In [59]:
from ibm_watsonx_ai import APIClient
import os
from dotenv import load_dotenv
from ibm_watsonx_ai.experiment import TuneExperiment


# Load Environment Variables
load_dotenv()

True

In [60]:
wml_credentials = {
    "url": os.getenv("WATSONX_URL"),            # WatsonX URL
    "apikey": os.getenv("WATSONX_API_KEY")
}

In [61]:
client = APIClient(
    credentials=wml_credentials,
    icp=False    # Important: IBM Cloud, not Cloud Pak
)

In [62]:
client.set.default_project(os.getenv("WATSONX_PROJECT_ID"))
project_id = "d0c2d56e-8ad1-49af-9e96-18a0648470ee"
print(project_id)

d0c2d56e-8ad1-49af-9e96-18a0648470ee


In [ ]:
model_id = "model_ff5659fd-d346-4220-82ff-69b5745242bc"

if 'model_id' in tuning_details.get('entity', {}):
    model_id = tuning_details['entity']['model_id']
model_id

'6e10d16e-f256-44baa77b-069fd2af6fdd'


In [19]:
from datetime import datetime
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "PT DEPLOYMENT SDK - IBM Granite - Fine tuned",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.SERVING_NAME : f"pt_sdk_deployment_{datetime.utcnow().strftime('%Y_%m_%d_%H%M%S')}"
}
deployment_details = client.deployments.create(model_id, meta_props)

C:\Users\USER\AppData\Local\Temp\ipykernel_21912\1900128606.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  client.deployments.ConfigurationMetaNames.SERVING_NAME : f"pt_sdk_deployment_{datetime.utcnow().strftime('%Y_%m_%d_%H%M%S')}"




######################################################################################

Synchronous deployment creation for id: '375301ab-a3f7-4aeb-9505-ab80cd081369' started

######################################################################################


initializing................
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='7303cc41-f661-4ba0-960f-9bb6fcb8d0a8'
-----------------------------------------------------------------------------------------------




In [ ]:
model_id_inference = deployment_details['metadata']['id']
model_id_inference

'375301ab-a3f7-4aeb-9505-ab80cd081369'


In [70]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

generate_params = {
    GenParams.TEMPERATURE: 0,
    GenParams.MAX_NEW_TOKENS: 1000,
    GenParams.DECODING_METHOD: "greedy",
}

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference

tuned_model = ModelInference(
    model_id=model_id_inference,
    params=generate_params,
    api_client=client
)

In [ ]:
system_prompt = "You are a leading expert in stakeholder management with extensive experience in complex, multi-stakeholder projects. Provide clear, strategic, and research-informed answers that address the challenges of managing diverse stakeholders across sectors, such as in collaborative initiatives involving industry, academia, and public institutions — for example, Research, Development, and Innovation (RDI) projects. Maintain a professional tone and focus on practical, actionable guidance."

# Questions
prompts = [
    "In large collaborative projects, how can organizations overcome the challenge of aligning different stakeholders with a common vision and strategy?",
    "In collaborative R&D projects, how can organizations prevent conflicts over intellectual property rights among stakeholders?",
    "How should stakeholder conflicts and unexpected changes be managed during project execution?",
    "How can project managers monitor stakeholder satisfaction and engagement throughout the life of a collaborative project?",
    "In collaborative R&D projects involving diverse types of stakeholders, how can organizations overcome the challenge of aligning different competencies, mindsets, and goals toward a common vision?",
    "How can collaborative R&D projects manage the challenge of high stakeholder turnover and resource changes during execution?",
    "How can communication between scientific and business stakeholders be improved in collaborative innovation projects?",
    "How is stakeholder prioritization managed in collaborative R&D projects?",
    "How can teams manage the misalignment of planning horizons and working styles between universities, public institutions, and companies in collaborative R&D?",
    "In collaborative R&D projects involving stakeholders from industry and research, how can organizations handle differences in goals, language, and working pace?",
    "How can stakeholder prioritization be handled in large collaborative projects with varying levels of involvement and risk?",
    "What should be done when stakeholders fail to deliver on agreed commitments during a collaborative project?",
    "How can project teams handle the challenge of keeping industry partners engaged and aligned in high-frequency collaborative workflows?",
    "How can documentation be standardized across diverse stakeholders in large-scale projects?"
]

results = []

for idx, user_question in enumerate(prompts, 1):
    full_prompt = f"{system_prompt}\n\nQuestion: {user_question}"

    response = tuned_model.generate_text(
        prompt=full_prompt
    )
    results.append({
        "Prompt_ID": idx,
        "Prompt": user_question,
        "Response": response
    })
    
    print(f"\n🔹 Prompt - id {idx}: {user_question}")
    print(f"📝 Response - id {idx}: {response}")
    print("-" * 1000)



🔹 Prompt - id 1: In large collaborative projects, how can organizations overcome the challenge of aligning different stakeholders with a common vision and strategy?
📝 Response - id 1: Answer: Develop a clear and concise project vision that articulates the desired outcomes and creates a common goal for all stakeholders. Provide regular updates on the project's progress and involve stakeholders in decision-making processes. Encourage open communication and listening across all stakeholder groups to ensure that their needs and concerns are heard and addressed.
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [82]:
import pandas as pd
from pathlib import Path

df = pd.DataFrame(results)
file_path = "Results_Fine_Tuned.xlsx"
df.to_excel(file_path, index=False)
